In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
with open('data/raw/frankenstein.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[200:400])

 evil
forebodings. I arrived here yesterday, and my first task is to assure
my dear sister of my welfare and increasing confidence in the success
of my undertaking.

I am already far north of London, 


In [4]:
chars = sorted(set(text))
vocab_size = len(chars)
learning_rate = 3e-4
max_iter = 2000
# print(chars)
# print(len(chars))

In [5]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([32, 51, 66, 66, 51, 64,  1,  9,  0,  0, 46, 39, 61,  1, 33, 64, 65,  7,
         1, 38, 47, 68, 55, 58, 58, 51,  5,  1, 25, 60, 53, 58, 47, 60, 50,  7,
        46,  0,  0,  0, 38, 66,  7,  1, 36, 51, 66, 51, 64, 65, 48, 67, 64, 53,
        54,  5,  1, 24, 51, 49,  7,  1,  9,  9, 66, 54,  5,  1,  9, 15, 78,  7,
         0,  0,  0, 43, 61, 67,  1, 69, 55, 58, 58,  1, 64, 51, 56, 61, 55, 49,
        51,  1, 66, 61,  1, 54, 51, 47, 64,  1])


In [7]:
n = int(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


In [44]:
def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    # print(ix)

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    # print("x:", x)
    # print("y:", y)

    return x, y

In [45]:
x, y = get_batch('train')

In [10]:
print(torch.cuda.device_count())

1


In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'test']:
        losses = torch.zeros(1000)
        for k in range(1000):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [13]:
model = BigramLanguageModel(vocab_size=vocab_size)

m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


RmDhA22rI5q
WVMgéêphjwd_]j[?]G-EFp;PwkBsMôoll-2DOVNw,JrDEbLua‘Jg2êdun,mgfHB!hE’D9
m’(gWf0ipuê7chT[gæ‘cpHèumN-—P8êfnYAaF;_x?
uhN9jm[:s6UTA4gtBD7RqDv”YSlL?kRC“BDi[mkRNMg6Gu:7.eoDhTJwHD,C‘MoGyb7L57aR y?V:(lC—zyG!ætPR5‘NæIywRbOHK8orôGyEYlgV;-x3j‘Gyiê)zè9[Rs;;bKNbKG9W,PxVêyiv?[S4aRhj3.V
-Vi]Eb’-!yx!jVP8eqRLnkRéP0g1a]BCêexdVPêê;KC‘x?ôfG
)?ôtVy3Péw1-a’VA“o?èæGS:”uæVwp ?BU;FèFpcI8HK!g4d_-YieMgDl:20æêkTeMC‘x am:P-.q
)êyPêe34 99.VP0mVS(-o.hê”‘?y?vVrp(hFie;“Lê!p7DEY)!a—me;Hs
M‘u_Vg;vsKS,J7nk4g—!Px:iêL” (!U


In [31]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for i in range(max_iter):
    if i % 500 == 0:
        losses = estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f}, test loss {losses['test']:.4f}")
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step 0: train loss 2.4566, test loss 2.4566
step 500: train loss 2.4657, test loss 2.4586
step 1000: train loss 2.4554, test loss 2.4541
step 1500: train loss 2.4624, test loss 2.4628
2.563657760620117


In [32]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


ns e pred thas tls ves, mmpad ave, cu; m te at
to bupig at Y-nered f
Alsmant houss meredssavoasomanepowancou pes tived if phadeand redestrer I, l;_jby g-
pre thothis, gnchanonguindea seasenapas in.’I my PC‘CNan hindoves
Bmisungelaren ticeanghod; as min, felitrene; “m tsindis
re br watand, th qRCin but p.’æ‘3HI
er cory foumethit thatow f thecoremom Soutle t Thit o h t okead owhencictang)avedsfrbaco d vorizcu to ie whe s f trt rne brs
Ind onchicho t akK”“ê)Jôjod herege
miêNARè0knsisuiad hlinccamec
